# Install Library

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Simple Training NER

In [ ]:
from simpletransformers.ner import NERModel, NERArgs

## Load pretrained (jelek)

In [ ]:
labels = ['B-CRD', 'B-DAT', 'B-EVT', 'B-FAC', 'B-GPE', 'B-LAN', 'B-LAW', 'B-LOC', 'B-MON', 'B-NOR', 'B-ORD', 'B-ORG', 'B-PER', 'B-PRC', 'B-PRD', 'B-QTY', 'B-REG', 'B-TIM', 'B-WOA', 'I-CRD', 'I-DAT', 'I-EVT', 'I-FAC', 'I-GPE', 'I-LAN', 'I-LAW', 'I-LOC', 'I-MON', 'I-NOR', 'I-ORD', 'I-ORG', 'I-PER', 'I-PRC', 'I-PRD', 'I-QTY', 'I-REG', 'I-TIM', 'I-WOA', 'O']

In [ ]:
# Configure the model args
model_args = NERArgs()
# model_args.use_cuda = False
model_args.labels_list = labels

In [ ]:
model = NERModel(
    "bert", "cahya/bert-base-indonesian-NER",  args=model_args
)

In [ ]:
# Make predictions with the model
texts = [
    "Gubernur Bank Indonesia Agus Martowardojo bersama jajaran deputi Gubernur Bank Indonesia menggelar konferensi pers usai Rapat Dewan Gubernur di Bank Indonesia, Jakarta, Kamis (17/5/2015)",
    "Selama 24 jam puncak mahameru di Malang kebanjiran pendaki dari Wina",
]
predictions, raw_outputs = model.predict(texts)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
len(predictions)

2

In [ ]:
predictions[0]

[{'Gubernur': 'O'},
 {'Bank': 'O'},
 {'Indonesia': 'O'},
 {'Agus': 'O'},
 {'Martowardojo': 'O'},
 {'bersama': 'O'},
 {'jajaran': 'O'},
 {'deputi': 'O'},
 {'Gubernur': 'O'},
 {'Bank': 'O'},
 {'Indonesia': 'O'},
 {'menggelar': 'O'},
 {'konferensi': 'B-EVT'},
 {'pers': 'I-EVT'},
 {'usai': 'O'},
 {'Rapat': 'O'},
 {'Dewan': 'O'},
 {'Gubernur': 'O'},
 {'di': 'O'},
 {'Bank': 'O'},
 {'Indonesia,': 'O'},
 {'Jakarta,': 'O'},
 {'Kamis': 'O'},
 {'(17/5/2015)': 'O'}]

## Train

In [ ]:
from simpletransformers.ner import NERModel, NERArgs
import pandas as pd
import logging
import sys

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
%cd /content/drive/MyDrive/nolimit/NER/simple_transformer/nergrit-corpus/ner/data
file_train_corrected = f'train_corrected.txt'
file_valid_corrected = f'valid_corrected.txt'
file_test_corrected = f'test_corrected.txt'
file_labels_map = f'labels-map.csv'

/content/drive/MyDrive/nolimit/NER/simple_transformer/nergrit-corpus/ner/data


In [ ]:
# Function to read ner file in connl format and return a DataFrame with columns: sentence_id, words, labels
def get_ner_data(filename):
    word_list = []
    sentence_counter = 0
    with open(filename) as fp:
        for cnt, line in enumerate(fp):
            try:
                texts = line.split()
                if len(texts) != 0:
                    word, label = ' '.join(texts[0:-1]), texts[-1]
                    word_list.append([sentence_counter, word, label])
                else:
                    sentence_counter += 1
            except:
                print("Unexpected error:", sys.exc_info()[0], cnt, line)
                word_list.append([sentence_counter, "", ""])
                sentence_counter += 1
                pass
    print(f'read {cnt} lines')
    ner_data = pd.DataFrame(word_list, columns=["sentence_id", "words", "labels"])
    return ner_data

In [ ]:
train_data = get_ner_data(file_train_corrected)
valid_data = get_ner_data(file_valid_corrected)
test_data = get_ner_data(file_test_corrected)
len(train_data),len(valid_data),len(test_data)

read 321756 lines
read 64207 lines
read 66972 lines


(309206, 61682, 64571)

In [ ]:
train_data.head(10)

,sentence_id,words,labels
0,0,Indonesia,B-GPE
1,0,mengekspor,O
2,0,produk,O
3,0,industri,O
4,0,skala,O
5,0,besar,O
6,0,ke,O
7,0,Amerika,B-GPE
8,0,Serikat,I-GPE
9,0,.,O


In [ ]:
valid_data.head(10)

,sentence_id,words,labels
0,0,Rupiah,O
1,0,Terus,O
2,0,Melemah,O
3,0,",",O
4,1,BI,B-ORG
5,1,:,O
6,1,Kebijakan,O
7,1,Moneter,O
8,1,Sudah,O
9,1,Tepat,O


In [ ]:
# The NERGRIT corpus contains 19 entities, each with Inside- and Beginning-Tag, plus an Outside-Tag. Alltogether become 39 categories. The entities have following meaning:

# 'CRD' --> Cardinal
# 'DAT' --> Date
# 'EVT' --> Event
# 'FAC' --> Facility
# 'GPE' --> Geopolitical Entity
# 'LAW' --> Law Entity (such as Undang-Undang)
# 'LOC' --> Location
# 'MON' --> Money
# 'NOR' --> Political Organization
# 'ORD' --> Ordinal
# 'ORG' --> Organization
# 'PER' --> Person
# 'PRC' --> Percent
# 'PRD' --> Product
# 'QTY' --> Quantity
# 'REG' --> Religion
# 'TIM' --> Time
# 'WOA' --> Work of Art
# 'LAN' --> Language

In [ ]:
df = pd.read_csv(file_labels_map, sep=' ', names=['X','Y'])
labels = list(set(df['Y']))
labels.sort()
len(labels)

39

In [ ]:
labels

['B-CRD',
 'B-DAT',
 'B-EVT',
 'B-FAC',
 'B-GPE',
 'B-LAN',
 'B-LAW',
 'B-LOC',
 'B-MON',
 'B-NOR',
 'B-ORD',
 'B-ORG',
 'B-PER',
 'B-PRC',
 'B-PRD',
 'B-QTY',
 'B-REG',
 'B-TIM',
 'B-WOA',
 'I-CRD',
 'I-DAT',
 'I-EVT',
 'I-FAC',
 'I-GPE',
 'I-LAN',
 'I-LAW',
 'I-LOC',
 'I-MON',
 'I-NOR',
 'I-ORD',
 'I-ORG',
 'I-PER',
 'I-PRC',
 'I-PRD',
 'I-QTY',
 'I-REG',
 'I-TIM',
 'I-WOA',
 'O']

### The Training with bert-base-indonesian-522M

In [ ]:
%cd /content/drive/MyDrive/nolimit/NER/simple_transformer/nergrit-corpus

/content/drive/MyDrive/nolimit/NER/simple_transformer/nergrit-corpus


In [ ]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 20
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.output_dir = '/output/ner/bert-base-indonesian'
model_args.best_model_dir = '/output/ner/bert-base-indonesian/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels
model_args.do_lower_case = True

In [ ]:
model_bert_base = NERModel(
    "bert", "cahya/bert-base-indonesian-522M", labels=labels, args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

In [ ]:
# Train the model
model_bert_base.train_model(train_data, eval_data=valid_data)

In [ ]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/315 [00:00<?, ?it/s]

In [ ]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(test_data)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
# Make predictions with the model
texts = [
    "Gubernur Bank Indonesia Agus Martowardojo bersama jajaran deputi Gubernur Bank Indonesia menggelar konferensi pers usai Rapat Dewan Gubernur di Bank Indonesia, Jakarta, Kamis (17/5/2015)",
    "Selama 24 jam puncak Mahameru di Malang kebanjiran pendaki dari Wina",
]

In [ ]:
texts = ["""JAKARTA, KOMPAS.com - Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng. Menurut Puan, Megawati bicara demikian karena dia melihat potensi besar di diri Jokowi. Mega tengah mengibaratkan PDI-P sebagai wadah bagi para kader, termasuk Jokowi, untuk berjuang. "Jadi ini sebagai wadah. Jadi maksudnya, ayo dong kita sama-sama maju, ayo dong sama-sama kita memberikan bakti kita, atau apa yang bisa kita lakukan untuk bisa membangun bangsa dan negara. Kan diperlukan, it takes two to tango untuk bisa melaksanakan hal itu," kata Puan dalam acara Rosi Kompas TV, dikutip Jumat (13/1/2023). Puan mengatakan, sebagai ketua umum PDI-P, Megawati selalu menempatkan diri sebagai ibu bagi kader-kader partai banteng. Dia pun menilai, pernyataan Mega itu wajar diucapkan seorang ketua umum partai. "Kenapa di acara PDI Perjuangan bicara seperti itu, karena ini acara internal, dianggap semua (kader) itu anak-anaknya," ujarnya. Puan mengatakan, Megawati tak bermaksud merendahkan Jokowi dengan mengatakan nasib Jokowi mungkin berbeda jika PDI-P tak ada. Sebaliknya, Ketua DPR RI itu bilang, ibundanya sangat menyayangi dan menghormati Jokowi. "Kalau tahu, Ibu Mega itu sayang banget sama Pak Jokowi. Dan tentu saja hormat sekali sama Pak Jokowi sebagai presiden," kata Puan. Puan lantas menyinggung pidato Megawati beberapa waktu lalu yang menyebut bahwa dia menangis karena Jokowi terus menerus dirundung. Mega bahkan mengaku siap pasang badan, tak apa dia dirundung asalkan bukan bukan Jokowi yang kena bully. Menurut Puan, sikap Megawati itu menunjukkan betapa dia sangat menyayangi dan menghormati Jokowi sebagai adiknya. Bahwa Megawati ingin Jokowi sukses menjalankan tugas-tugasnya sebagai kepala negara. "Saya baru pertama kali ini mendengar Bu Mega membela seseorang sampai seperti itu," ucap Puan. "Bahkan (Megawati) sampai (mengatakan), nggak papa saya di- bully karena saya mau membela Pak Jokowi. Itu kalau bukan sayang dan hormat apa," tuturnya. Lagi pula, lanjut Puan, dalam pidatonya Megawati juga sempat memuji Jokowi dengan menyebut bahwa mantan Gubernur DKI Jakarta itu pintar sehingga PDI-P bersedia mengusungnya jadi pemimpin. Oleh karenanya, Puan menegaskan bahwa Megawati tak bermaksud buruk, apalagi menghina Jokowi lewat pidatonya. "Jadi bukan ada yang rasa yang mau merendahkan atau sepertinya nggak hormat, itu justru kebalikan," tutur mantan Menteri Koordinator Bidang Pembangunan Manusia dan Kebudayaan (Menko PMK) itu. Sebelumnya, nama Presiden Jokowi berulang kali disinggung oleh Megawati dalam pidatonya di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang digelar di Jakarta International Expo, Kemayoran, Jakarta Pusat, Selasa (10/1/2023). Di hadapan ribuan kader yang hadir, Megawati berkelakar, nasib Jokowi tidak akan sama seperti sekarang jika tak ada PDI-P. "Pak Jokowi itu kayak begitu lho, mentang-mentang. Lah iya, padahal Pak Jokowi kalau enggak ada PDI Perjuangan juga, aduh, kasihan dah," kata Megawati sambil tertawa. Seloroh Megawati tersebut disambut tawa para tamu undangan, termasuk Jokowi yang duduk di barisan kursi tamu terdepan. Presiden kelima RI itu bilang, seandainya PDI-P tak memberikan dukungan, maka Jokowi tidak akan jadi presiden seperti sekarang. "Lho legal formal lho, beliau jadi presiden itu enggak ada kan ini, legal formal diikuti terus sama saya," ujarnya. Pernyataan Megawati itu pun memunculkan kritik dari sejumlah kalangan. Putri Soekarno itu bahkan dianggap merendahkan presiden."""]

In [ ]:
# Make predictions with the model
texts = [
    "Gubernur Bank Indonesia Agus Martowardojo bersama jajaran deputi Gubernur Bank Indonesia menggelar konferensi pers usai Rapat Dewan Gubernur di Bank Indonesia, Jakarta, Kamis (17/5/2015)",
    "Selama 24 jam puncak Mahameru di Malang kebanjiran pendaki dari Wina",
]

predictions, raw_outputs = model_bert_base.predict(texts)
predictions

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'JAKARTA,': 'B-GPE'},
  {'KOMPAS.com': 'B-ORG'},
  {'-': 'O'},
  {'Ketua': 'B-NOR'},
  {'DPP': 'I-NOR'},
  {'PDI': 'I-NOR'},
  {'Perjuangan': 'I-NOR'},
  {'Puan': 'B-PER'},
  {'Maharani': 'I-PER'},
  {'menjelaskan': 'O'},
  {'maksud': 'O'},
  {'dari': 'O'},
  {'pidato': 'O'},
  {'Megawati': 'B-PER'},
  {'Soekarnoputri': 'I-PER'},
  {'di': 'O'},
  {'acara': 'O'},
  {'Hari': 'B-EVT'},
  {'Ulang': 'I-EVT'},
  {'Tahun': 'I-EVT'},
  {'(HUT)': 'I-EVT'},
  {'ke-50': 'I-EVT'},
  {'PDI-P': 'B-NOR'},
  {'yang': 'O'},
  {'menyinggung': 'O'},
  {'nasib': 'O'},
  {'Presiden': 'O'},
  {'Joko': 'B-PER'},
  {'Widodo': 'I-PER'},
  {'jika': 'O'},
  {'tak': 'O'},
  {'ada': 'O'},
  {'partai': 'B-NOR'},
  {'banteng.': 'I-NOR'},
  {'Menurut': 'O'},
  {'Puan,': 'B-PER'},
  {'Megawati': 'B-PER'},
  {'bicara': 'O'},
  {'demikian': 'O'},
  {'karena': 'O'},
  {'dia': 'O'},
  {'melihat': 'O'},
  {'potensi': 'O'},
  {'besar': 'O'},
  {'di': 'O'},
  {'diri': 'O'},
  {'Jokowi.': 'B-PER'},
  {'Mega': 'B-PER'},
  {

In [ ]:
# save the model to disk

import pickle

filename = '/content/drive/MyDrive/nolimit/NER/simple_transformer/model_bert_base.pkl'
pickle.dump(model_bert_base, open(filename, 'wb'))

## Load model (hasil train)

In [ ]:
import pickle

model = pickle.load(open('/content/drive/MyDrive/nolimit/NER/simple_transformer/model_simple_ner_2.pkl', 'rb'))

In [ ]:
model

In [ ]:
# Make predictions with the model
texts = [
    "Gubernur Bank Indonesia Agus Martowardojo bersama jajaran deputi Gubernur Bank Indonesia menggelar konferensi pers usai Rapat Dewan Gubernur di Bank Indonesia, Jakarta, Kamis (17/5/2015)"
]

predictions, raw_outputs = model.predict(texts)
predictions

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Gubernur': 'B-NOR'},
  {'Bank': 'I-NOR'},
  {'Indonesia': 'I-NOR'},
  {'Agus': 'B-PER'},
  {'Martowardojo': 'I-PER'},
  {'bersama': 'O'},
  {'jajaran': 'O'},
  {'deputi': 'B-NOR'},
  {'Gubernur': 'I-NOR'},
  {'Bank': 'I-NOR'},
  {'Indonesia': 'I-NOR'},
  {'menggelar': 'O'},
  {'konferensi': 'B-EVT'},
  {'pers': 'I-EVT'},
  {'usai': 'O'},
  {'Rapat': 'B-EVT'},
  {'Dewan': 'I-EVT'},
  {'Gubernur': 'I-EVT'},
  {'di': 'O'},
  {'Bank': 'B-ORG'},
  {'Indonesia,': 'I-ORG'},
  {'Jakarta,': 'B-GPE'},
  {'Kamis': 'B-DAT'},
  {'(17/5/2015)': 'I-DAT'}]]

## **text_mentah**

In [ ]:
text_mentah = """tirto.id - Sekjen DPP PDIP, Hasto Kristiyanto menyebut, dukungan PAN kepada Ganjar Pranowo merupakan bentuk pergeseran substansi demokrasi. Menurut Hasto, demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.

“Banyak motif dalam mencalonkan seseorang. Ada yang bermotif untuk mendongkrak elektoral partai tersebut. Ada juga motif sebagai jalan pintas karena tidak melakukan kaderisasi di internal partai," kata Hasto di Sekolah Partai PDIP pada Kamis (2/3/2023).

Karena itu, Hasto menyayangkan tindakan PAN tersebut, dan berharap partai politik di Indonesia bisa mengusung kadernya sendiri. Sehingga pendidikan di internal partai berjalan dengan baik.

“Tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling rasional," ujarnya.

Hasto juga mengklaim dalam proses dukungan Ganjar ketika pidato Ketua Umum PAN, Zulkifli Hasan dalam acara Rakornas di Semarang pada pada Minggu (26/2/2023) tak ada komunikasi dengan pihaknya. Menurutnya komunikasi dengan PAN tidak pernah membahas soal nakal capres atau cawapres.

"Ya kalau komunikasi dilakukan untuk hal-hal lain. Di luar hal yang terkait dengan pencalonan presiden dan wakil presiden," jelasnya.

Selain itu, Hasto juga membantah bahwa Presiden Joko Widodo mengamini dukungan kepada Ganjar Pranowo tersebut. Walaupun saat itu, Jokowi hadir sebagai tamu dan menjadi pembicara. Namun Jokowi tak ikut campur soal substansi acara.

"Tetapi terkait dengan agenda di acara internal partai, Pak Jokowi tidak pernah ikut campur. PDI Perjuangan juga tidak pernah ikut campur pada setiap agenda partai," terangnya."""

In [ ]:
import nltk
import re
import string
nltk.download('punkt')
nltk.download('stopwords')

from nltk.util import ngrams

from nltk.corpus import stopwords
sw = stopwords.words("indonesian")

def text_cleaning(text, remove_stop_words):
    # Clean the text

    text = text.lower() #lowercase atau case folding
    text = re.sub('@[^\s]+', ' ', text) #remove username
    text = re.sub('\[.*?\]', ' ', text) # remove square brackets
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', text) # remove URLs
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text) # remove punctuation
    # text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub("[!@#$%^&*()[]{};:,./<>?\|`~-=_+–]", ' ', text)
    text = re.sub("–", ' ', text)

    # Optionally, remove stop words
    if remove_stop_words:
        # load stopwords
        text = text.split()
        text = [w for w in text if not w in sw] #6
        text = " ".join(text)
        text = text.lower()

    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Predict Class

In [ ]:
import re
import pandas as pd

In [ ]:
# new_text = re.sub(r".*- ", "", text_mentah)
# print(new_text)

new_text = text_mentah

sentences = []
start = 0
for match in re.finditer(r'\n', new_text):
    end = match.start()
    sentences.append(new_text[start:end])
    start = match.end()

# add the last sentence
sentences.append(new_text[start:])
sentences.remove(sentences[-1])
print(sentences)
print(len(sentences))

['tirto.id - Sekjen DPP PDIP, Hasto Kristiyanto menyebut, dukungan PAN kepada Ganjar Pranowo merupakan bentuk pergeseran substansi demokrasi. Menurut Hasto, demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.', '', '“Banyak motif dalam mencalonkan seseorang. Ada yang bermotif untuk mendongkrak elektoral partai tersebut. Ada juga motif sebagai jalan pintas karena tidak melakukan kaderisasi di internal partai," kata Hasto di Sekolah Partai PDIP pada Kamis (2/3/2023).', '', 'Karena itu, Hasto menyayangkan tindakan PAN tersebut, dan berharap partai politik di Indonesia bisa mengusung kadernya sendiri. Sehingga pendidikan di internal partai berjalan dengan baik.', '', '“Tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling rasional," ujarnya.', '', 'Hasto juga mengklaim dalam proses dukungan Ganjar ketika pidato Ketua Umum PAN, Zulkifli Hasa

In [ ]:
# def clean_list_of_text(text_list):
#     return list(map(lambda x: text_cleaning(x, False), text_list))

# # apply the text cleaning function to the list
# sentences = clean_list_of_text(sentences)

In [ ]:
print(sentences)

['tirto.id - Sekjen DPP PDIP, Hasto Kristiyanto menyebut, dukungan PAN kepada Ganjar Pranowo merupakan bentuk pergeseran substansi demokrasi. Menurut Hasto, demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.', '', '“Banyak motif dalam mencalonkan seseorang. Ada yang bermotif untuk mendongkrak elektoral partai tersebut. Ada juga motif sebagai jalan pintas karena tidak melakukan kaderisasi di internal partai," kata Hasto di Sekolah Partai PDIP pada Kamis (2/3/2023).', '', 'Karena itu, Hasto menyayangkan tindakan PAN tersebut, dan berharap partai politik di Indonesia bisa mengusung kadernya sendiri. Sehingga pendidikan di internal partai berjalan dengan baik.', '', '“Tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling rasional," ujarnya.', '', 'Hasto juga mengklaim dalam proses dukungan Ganjar ketika pidato Ketua Umum PAN, Zulkifli Hasa

In [ ]:
  for i, y in enumerate(sentences):
    if len(y) < 10:
      sentences.remove(sentences[i])
    else:
      pass

  for i, y in enumerate(sentences):
    print(i, '-', y)

0 - tirto.id - Sekjen DPP PDIP, Hasto Kristiyanto menyebut, dukungan PAN kepada Ganjar Pranowo merupakan bentuk pergeseran substansi demokrasi. Menurut Hasto, demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.
1 - “Banyak motif dalam mencalonkan seseorang. Ada yang bermotif untuk mendongkrak elektoral partai tersebut. Ada juga motif sebagai jalan pintas karena tidak melakukan kaderisasi di internal partai," kata Hasto di Sekolah Partai PDIP pada Kamis (2/3/2023).
2 - Karena itu, Hasto menyayangkan tindakan PAN tersebut, dan berharap partai politik di Indonesia bisa mengusung kadernya sendiri. Sehingga pendidikan di internal partai berjalan dengan baik.
3 - “Tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling rasional," ujarnya.
4 - Hasto juga mengklaim dalam proses dukungan Ganjar ketika pidato Ketua Umum PAN, Zulkifli Hasan dalam ac

In [ ]:
# sentences = """
# Garut: Gempa berkekuatan magnitudo 4,3 pada Rabu, 1 Februari pukul 22.57 WIB telah menyebabkan 495 rumah mengalami kerusakan tepatnya berada di Kecamatan Pasirwangi dan Kecamatan Samarang, Kabupaten Garut. Kejadian tersebut tidak menyebabkan korban jiwa tetapi, petugas gabungan TNI, Polri dan BPBD masih terus melakukan pendataan. Berdasarkan data laporan BPBD Kabupaten Garut, hingga Kamis malam tidak ada korban jiwa. Tercatat total rumah terdampak sebanyak 495 unit rumah, terdiri dari 367 unit rumah mengalami rusak ringan (RR), 111 unit rumah rusak sedang (RS) dan 17 unit rumah rusak berat, serta 8 unit fasilitas pendidikan mengalami kerusakan. Kepala Pelaksana Badan Penanggulangan Bencana Daerah Kabupaten Garut, Satria Budi mengatakan, gempa bumi berkekuatan 4,3 magnitudo terletak pada koordinat 7.27 LS dan 107.73 BT, atau tepatnya berlokasi di darat pada jarak 19 kilometer Barat Daya Kabupaten Garut pada kedalaman 3 kilometer. "Berdasarkan rilis BMKG, dampak gempa bumi yang digambarkan peta tingkat guncangan (Shakemap) BMKG dan berdasarkan laporan dari masyarakat dirasakan di wilayah GarutÂ Pasirwangi, Cisurupan, Bayongbong, Samarang dengan Skala Intensitas IV MMI. Dengan memperhatikan lokasi episenter dan kedalaman hiposenternya, gempabumi yang terjadi merupakan jenis gempabumi dangkal akibat aktivitas Sesar Garsela," katanya, Jumat, 3 Februari 2023. Sementara Wakil Bupati Garut, Helmi Budiman meminta masyarakat untuk tetap tenang. Petugas gabungan TNI, Polri dan BPBD sedang melakukan verifikasi di lapangan. "Secara umum untuk di Kecamatan Samarang lebih sedikit tapi kondisinya berat dan sedang, tapi di Pasirwangi jumlahnya memang banyak 370 tapi ringan dan sedang. Jadi, sekarang ini sedang melakukan verifikasi dulu dan personil yang melakukan assessment di lapangan, kita panggil di BPBD dan nanti akan melakukan pembahasan sehingga nanti tindaklanjut akan dilakukan bisa semakin jelas," ujarnya. Ia berharap masyarakat yang rumahnya mengalami rusak berat untuk sementara waktu supaya berpindah ke lokasi lebih aman, karena ditakutkan gempa susulan yang terjadi. "Jadi, harap tenang dan siap-siap ya kalau ada gempa susulan sekali lagi rumah yang rusak berat lah ya untuk tidak ditinggali dulu, dan tentu yang bisa kita lakukan yang terbaik itu lakukanlah, bisa di keluarga, bisa di tetangga atau di tempat yang lain yang aman," paparnya.
# """
# # sentences = text_cleaning(sentences, False)

In [ ]:
# predict

# Membuat kolom baru
df = pd.DataFrame(columns=["words", "label"])

predictions, raw_outputs = model.predict(sentences)

# Looping untuk membaca data dan memasukkan ke dalam dataframe
for i in predictions:
    text = []
    label = []
    for j in i:
        text.append(list(j.keys())[0])
        label.append(list(j.values())[0])
    df = df.append({"words": text, "label": label}, ignore_index=True)

word = [item for sublist in df['words'].tolist() for item in sublist]
label = [item for sublist in df['label'].tolist() for item in sublist]

hasil = pd.DataFrame({'words': word, 'label': label})

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
hasil

,words,label
0,tirto.id,B-ORG
1,-,O
2,Sekjen,B-NOR
3,DPP,I-NOR
4,"PDIP,",I-NOR
...,...,...
204,ikut,O
205,campur,O
206,soal,O
207,substansi,O


In [ ]:
# kalau udah dapat hasil di df nya maka kesini

hasil_tanpaO = hasil.drop_duplicates().reset_index(drop=True)
hasil_tanpaO = hasil[hasil['label'] != 'O'].reset_index(drop=True)

In [ ]:
# hapus titik dan koma pada kolom text
hasil['words'] = hasil['words'].str.replace('.com', '', regex=True)
hasil['words'] = hasil['words'].str.replace('[.,()-:;"â”]', '', regex=True)

In [ ]:
hasil

,words,label
0,tirtoid,B-ORG
1,,O
2,Sekjen,B-NOR
3,DPP,I-NOR
4,PDIP,I-NOR
...,...,...
204,ikut,O
205,campur,O
206,soal,O
207,substansi,O


In [ ]:
hasil['words'] = hasil['words'].str.lower()

In [ ]:
indeks = hasil[(hasil['label'] == 'B-PER') & (hasil['label'].shift(-1) != 'I-PER') & (hasil['words'].str.len() <= 3)].index
hasil = hasil.drop(indeks).reset_index(drop=True)

In [ ]:
hasil

,words,label
0,tirtoid,B-ORG
1,,O
2,sekjen,B-NOR
3,dpp,I-NOR
4,pdip,I-NOR
...,...,...
204,ikut,O
205,campur,O
206,soal,O
207,substansi,O


In [ ]:
# 'CRD' --> Cardinal
# 'DAT' --> Date
# 'EVT' --> Event
# 'FAC' --> Facility
# 'GPE' --> Geopolitical Entity
# 'LAW' --> Law Entity (such as Undang-Undang)
# 'LOC' --> Location
# 'MON' --> Money
# 'NOR' --> Political Organization
# 'ORD' --> Ordinal
# 'ORG' --> Organization
# 'PER' --> Person
# 'PRC' --> Percent
# 'PRD' --> Product
# 'QTY' --> Quantity
# 'REG' --> Religion
# 'TIM' --> Time
# 'WOA' --> Work of Art
# 'LAN' --> Language

In [ ]:
# ['B-CRD','B-DAT','B-EVT','B-FAC','B-GPE','B-LAN','B-LAW','B-LOC','B-MON','B-NOR',
#  'B-ORD','B-ORG','B-PER','B-PRC','B-PRD','B-QTY','B-REG','B-TIM','B-WOA',
#  'I-CRD','I-DAT','I-EVT','I-FAC','I-GPE','I-LAN','I-LAW','I-LOC','I-MON','I-NOR',
#  'I-ORD','I-ORG','I-PER','I-PRC','I-PRD','I-QTY','I-REG','I-TIM','I-WOA',
#  'O']

In [ ]:
# hasil = hasil.drop_duplicates().reset_index(drop=True)

In [ ]:
hasil.label.value_counts()

O        171
B-PER     13
B-NOR      6
I-PER      5
I-NOR      4
B-ORG      2
B-DAT      2
I-DAT      2
B-GPE      2
B-EVT      1
I-ORG      1
Name: label, dtype: int64

In [ ]:
# hasil[(hasil['label'] == 'B-GPE') | (hasil['label'] == 'I-GPE')]

In [ ]:
#full dibawah ini yang combine B-I

# print(text_mentah)
print('-'*100)

output = {}

for i in range(len(hasil)):

  # --------------- PERSON -----------------------------
  try:
    if hasil['label'][i]=='B-PER' and hasil['label'][i+1]!='I-PER':
      print(hasil['words'][i], '=> PERSON')
      output[hasil['words'][i]] = 'PERSON'
    else:
      pass

  except:
    pass

  try:
    if hasil['label'][i]=='B-PER' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> PERSON')
      output[hasil['words'][i]] = 'PERSON'

    else:
      pass

  except:
    pass

  try:
    if hasil['label'][i]=='B-PER' and hasil['label'][i+1]=='I-PER' and hasil['label'][i+2]=='I-PER':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> PERSON')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2]] = 'PERSON'
    elif hasil['label'][i]=='B-PER' and hasil['label'][i+1]=='I-PER':
      print(hasil['words'][i],hasil['words'][i+1], '=> PERSON')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1]] = 'PERSON'
    else:
      pass

  except:
    pass

  # --------------- LOCATION -----------------------------
  try:
    if hasil['label'][i]=='B-LOC' and hasil['label'][i+1]!='I-LOC':
      print(hasil['words'][i], '=> LOCATION')
      output[hasil['words'][i]] = 'LOCATION'
    else:
      pass

  except:
    if hasil['label'][i]=='B-LOC' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> LOCATION')
      output[hasil['words'][i]] = 'LOCATION'
    else:
      pass

  try:
    if hasil['label'][i]=='B-LOC' and hasil['label'][i+1]=='I-LOC' and hasil['label'][i+2]=='I-LOC':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> LOCATION')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2]] = 'LOCATION'
    elif hasil['label'][i]=='B-LOC' and hasil['label'][i+1]=='I-LOC':
      print(hasil['words'][i],hasil['words'][i+1], '=> LOCATION')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1]] = 'LOCATION'
    else:
      pass

  except:
    pass

  # --------------- ORGANIZATION ----------------------------
  try:
    if hasil['label'][i]=='B-ORG' and hasil['label'][i+1]!='I-ORG':
      print(hasil['words'][i], '=> ORGANIZATION')
      output[hasil['words'][i]] = 'ORGANIZATION'
    else:
      pass

  except:
    if hasil['label'][i]=='B-ORG' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> ORGANIZATION')
      output[hasil['words'][i]] = 'ORGANIZATION'
    else:
      pass

  try:
    if hasil['label'][i]=='B-ORG' and hasil['label'][i+1]=='I-ORG' and hasil['label'][i+2]=='I-ORG':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> ORGANIZATION')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2]] = 'ORGANIZATION'
    elif hasil['label'][i]=='B-ORG' and hasil['label'][i+1]=='I-ORG':
      print(hasil['words'][i],hasil['words'][i+1], '=> ORGANIZATION')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1]] = 'ORGANIZATION'
    else:
      pass

  except:
    pass

  # --------------- POLITICAL ORGANIZATION ----------------------------
  try:
    if hasil['label'][i]=='B-NOR' and hasil['label'][i+1]!='I-NOR':
      print(hasil['words'][i], '=> POLITICAL ORGANIZATION')
      output[hasil['words'][i]] = 'POLITICAL ORGANIZATION'
    else:
      pass

  except:
    if hasil['label'][i]=='B-NOR' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> POLITICAL ORGANIZATION')
      output[hasil['words'][i]] = 'POLITICAL ORGANIZATION'
    else:
      pass

  try:
    if hasil['label'][i]=='B-NOR' and hasil['label'][i+1]=='I-NOR' and hasil['label'][i+2]=='I-NOR':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> POLITICAL ORGANIZATION')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2]] = 'POLITICAL ORGANIZATION'
    else:
      pass

  except:
    if hasil['label'][i]=='B-NOR' and hasil['label'][i+1]=='I-NOR':
      print(hasil['words'][i],hasil['words'][i+1], '=> POLITICAL ORGANIZATION')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1]] = 'POLITICAL ORGANIZATION'
    else:
      pass

  # --------------- GEOPOLITICAL ENTITY ----------------------------
  try:
    if hasil['label'][i]=='B-GPE' and hasil['label'][i+1]!='I-GPE':
      print(hasil['words'][i], '=> GEOPOLITICAL ENTITY')
      output[hasil['words'][i]] = 'GEOPOLITICAL ENTITY'
    else:
      pass

  except:
    if hasil['label'][i]=='B-GPE' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> GEOPOLITICAL ENTITY')
      output[hasil['words'][i]] = 'GEOPOLITICAL ENTITY'
    else:
      pass

  try:
    if hasil['label'][i]=='B-GPE' and hasil['label'][i+1]=='I-GPE' and hasil['label'][i+2]=='I-GPE':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> GEOPOLITICAL ENTITY')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2]] = 'GEOPOLITICAL ENTITY'
    elif hasil['label'][i]=='B-GPE' and hasil['label'][i+1]=='I-GPE':
      print(hasil['words'][i],hasil['words'][i+1], '=> GEOPOLITICAL ENTITY')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1]] = 'GEOPOLITICAL ENTITY'
    else:
      pass

  except:
    pass


  # --------------- EVENT ----------------------------
  try:
    if hasil['label'][i]=='B-EVT' and hasil['label'][i+1]!='I-EVT':
      print(hasil['words'][i], '=> EVENT')
      output[hasil['words'][i]] = 'EVENT'
    else:
      pass

  except:
    if hasil['label'][i]=='B-EVT' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> EVENT')
      output[hasil['words'][i]] = 'EVENT'
    else:
      pass

  try:
    if hasil['label'][i]=='B-EVT' and hasil['label'][i+4]=='I-EVT' and hasil['label'][i+5]=='I-EVT':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2],hasil['words'][i+3],hasil['words'][i+4],hasil['words'][i+5], '=> EVENT')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2] + ' ' + hasil['words'][i+3] + ' ' + hasil['words'][i+4] + ' ' + hasil['words'][i+5]] = 'EVENT'
    elif hasil['label'][i]=='B-EVT' and hasil['label'][i+3]=='I-EVT' and hasil['label'][i+4]=='I-EVT':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2],hasil['words'][i+3],hasil['words'][i+4], '=> EVENT')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2] + ' ' + hasil['words'][i+3] + ' ' + hasil['words'][i+4]] = 'EVENT'
    elif hasil['label'][i]=='B-EVT' and hasil['label'][i+2]=='I-EVT' and hasil['label'][i+3]=='I-EVT':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2],hasil['words'][i+3], '=> EVENT')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2] + ' ' + hasil['words'][i+3]] = 'EVENT'
    elif hasil['label'][i]=='B-EVT' and hasil['label'][i+1]=='I-EVT' and hasil['label'][i+2]=='I-EVT':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> EVENT')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2]] = 'EVENT'
    elif hasil['label'][i]=='B-EVT' and hasil['label'][i+1]=='I-EVT':
      print(hasil['words'][i],hasil['words'][i+1], '=> EVENT')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1]] = 'EVENT'
    else:
      pass

  except:
    pass

  try:
    if hasil['label'][i]=='B-EVT' and hasil['label'][i+2]=='I-EVT' and hasil['label'][i+3]=='I-EVT':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2],hasil['words'][i+3], '=> EVENT')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2] + ' ' + hasil['words'][i+3]] = 'EVENT'
    elif hasil['label'][i]=='B-EVT' and hasil['label'][i+1]=='I-EVT':
      print(hasil['words'][i],hasil['words'][i+1], '=> EVENT')
      output[hasil['words'][i] + ' ' + hasil['words'][i+1]] = 'EVENT'
    else:
      pass

  except:
    pass


  # --------------- O -----------------------------
  try:
    if hasil['label'][i]=='O':
      print(hasil['words'][i], '=> O')
      output[hasil['words'][i]] = 'O'
    else:
      pass

  except:
    pass

----------------------------------------------------------------------------------------------------
tirtoid => ORGANIZATION
 => O
sekjen dpp pdip => POLITICAL ORGANIZATION
hasto kristiyanto => PERSON
menyebut => O
dukungan => O
pan => POLITICAL ORGANIZATION
kepada => O
ganjar pranowo => PERSON
merupakan => O
bentuk => O
pergeseran => O
substansi => O
demokrasi => O
menurut => O
hasto => PERSON
demokrasi => O
yang => O
baik => O
adalah => O
mendidik => O
kader => O
sendiri => O
dan => O
mengusungnya => O
untuk => O
pilpres => EVENT
dibandingkan => O
mengambil => O
kader => O
partai => O
lain => O
demi => O
meningkatkan => O
suara => O
“banyak => O
motif => O
dalam => O
mencalonkan => O
seseorang => O
ada => O
yang => O
bermotif => O
untuk => O
mendongkrak => O
elektoral => O
partai => O
tersebut => O
ada => O
juga => O
motif => O
sebagai => O
jalan => O
pintas => O
karena => O
tidak => O
melakukan => O
kaderisasi => O
di => O
internal => O
partai => O
kata => O
hasto => PERSON
di => O


In [ ]:
output.items()

dict_items([('tirtoid', 'ORGANIZATION'), ('', 'O'), ('sekjen dpp pdip', 'POLITICAL ORGANIZATION'), ('hasto kristiyanto', 'PERSON'), ('menyebut', 'O'), ('dukungan', 'O'), ('pan', 'POLITICAL ORGANIZATION'), ('kepada', 'O'), ('ganjar pranowo', 'PERSON'), ('merupakan', 'O'), ('bentuk', 'O'), ('pergeseran', 'O'), ('substansi', 'O'), ('demokrasi', 'O'), ('menurut', 'O'), ('hasto', 'PERSON'), ('yang', 'O'), ('baik', 'O'), ('adalah', 'O'), ('mendidik', 'O'), ('kader', 'O'), ('sendiri', 'O'), ('dan', 'O'), ('mengusungnya', 'O'), ('untuk', 'O'), ('pilpres', 'EVENT'), ('dibandingkan', 'O'), ('mengambil', 'O'), ('partai', 'O'), ('lain', 'O'), ('demi', 'O'), ('meningkatkan', 'O'), ('suara', 'O'), ('“banyak', 'O'), ('motif', 'O'), ('dalam', 'O'), ('mencalonkan', 'O'), ('seseorang', 'O'), ('ada', 'O'), ('bermotif', 'O'), ('mendongkrak', 'O'), ('elektoral', 'O'), ('tersebut', 'O'), ('juga', 'O'), ('sebagai', 'O'), ('jalan', 'O'), ('pintas', 'O'), ('karena', 'O'), ('tidak', 'O'), ('melakukan', 'O'), ('

In [ ]:
output

{'tirtoid': 'ORGANIZATION',
 '': 'O',
 'sekjen dpp pdip': 'POLITICAL ORGANIZATION',
 'hasto kristiyanto': 'PERSON',
 'menyebut': 'O',
 'dukungan': 'O',
 'pan': 'POLITICAL ORGANIZATION',
 'kepada': 'O',
 'ganjar pranowo': 'PERSON',
 'merupakan': 'O',
 'bentuk': 'O',
 'pergeseran': 'O',
 'substansi': 'O',
 'demokrasi': 'O',
 'menurut': 'O',
 'hasto': 'PERSON',
 'yang': 'O',
 'baik': 'O',
 'adalah': 'O',
 'mendidik': 'O',
 'kader': 'O',
 'sendiri': 'O',
 'dan': 'O',
 'mengusungnya': 'O',
 'untuk': 'O',
 'pilpres': 'EVENT',
 'dibandingkan': 'O',
 'mengambil': 'O',
 'partai': 'O',
 'lain': 'O',
 'demi': 'O',
 'meningkatkan': 'O',
 'suara': 'O',
 '“banyak': 'O',
 'motif': 'O',
 'dalam': 'O',
 'mencalonkan': 'O',
 'seseorang': 'O',
 'ada': 'O',
 'bermotif': 'O',
 'mendongkrak': 'O',
 'elektoral': 'O',
 'tersebut': 'O',
 'juga': 'O',
 'sebagai': 'O',
 'jalan': 'O',
 'pintas': 'O',
 'karena': 'O',
 'tidak': 'O',
 'melakukan': 'O',
 'kaderisasi': 'O',
 'di': 'O',
 'internal': 'O',
 'kata': 'O',


### Hasil

In [ ]:
new_output = {}
for k,v in output.items():
  if type(k) == tuple:
    new_output[' '.join(k)] = v

  else:
    new_output[''.join(k)] = v

# new_output

df = pd.DataFrame()

df['text'] = new_output.keys()
df['label'] = new_output.values()

In [ ]:
print('PERSON : ')
print(df[df['label'] == 'PERSON'])
print('-'*100)

print('POLITICAL ORGANIZATION : ')
print(df[df['label'] == 'POLITICAL ORGANIZATION'])
print('-'*100)

print('ORGANIZATION : ')
print(df[df['label'] == 'ORGANIZATION'])
print('-'*100)

print('GEOPOLITICAL ENTITY : ')
print(df[df['label'] == 'GEOPOLITICAL ENTITY'])
print('-'*100)

print('LOCATION : ')
print(df[df['label'] == 'LOCATION'])
print('-'*100)

PERSON : 
                  text   label
3    hasto kristiyanto  PERSON
8       ganjar pranowo  PERSON
15               hasto  PERSON
82              ganjar  PERSON
85      zulkifli hasan  PERSON
113        joko widodo  PERSON
117             jokowi  PERSON
----------------------------------------------------------------------------------------------------
POLITICAL ORGANIZATION : 
              text                   label
2  sekjen dpp pdip  POLITICAL ORGANIZATION
6              pan  POLITICAL ORGANIZATION
----------------------------------------------------------------------------------------------------
ORGANIZATION : 
              text         label
0          tirtoid  ORGANIZATION
54  sekolah partai  ORGANIZATION
----------------------------------------------------------------------------------------------------
GEOPOLITICAL ENTITY : 
         text                label
61  indonesia  GEOPOLITICAL ENTITY
88   semarang  GEOPOLITICAL ENTITY
-----------------------------------------

# Statement

**bismillah dibawah bisa untuk statement**

## input artikel

In [ ]:
import re
import pandas as pd

In [ ]:
sentences

['sekjen dpp pdip, hasto kristiyanto menyebut, dukungan pan kepada ganjar pranowo merupakan bentuk pergeseran substansi demokrasi. menurut hasto, demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.',
 '“banyak motif dalam mencalonkan seseorang. ada yang bermotif untuk mendongkrak elektoral partai tersebut. ada juga motif sebagai jalan pintas karena tidak melakukan kaderisasi di internal partai," kata hasto di sekolah partai pdip pada kamis (2/3/2023).',
 'karena itu, hasto menyayangkan tindakan pan tersebut, dan berharap partai politik di indonesia bisa mengusung kadernya sendiri. sehingga pendidikan di internal partai berjalan dengan baik.',
 '“tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling rasional," ujarnya.',
 'hasto juga mengklaim dalam proses dukungan ganjar ketika pidato ketua umum pan, zulkifli hasan dalam acara rakornas 

In [ ]:
# predict

def predict_person(string):
  global hasil
  global output
  global df
  global new_output
  global people
  # Membuat kolom baru
  df = pd.DataFrame(columns=["words", "label"])

  predictions, raw_outputs = model.predict([string])

  # print(predictions)

  # Looping untuk membaca data dan memasukkan ke dalam dataframe
  for i in predictions:
      text = []
      label = []
      for j in i:
          text.append(list(j.keys())[0])
          label.append(list(j.values())[0])
      df = df.append({"words": text, "label": label}, ignore_index=True)

  word = [item for sublist in df['words'].tolist() for item in sublist]
  label = [item for sublist in df['label'].tolist() for item in sublist]

  hasil = pd.DataFrame({'words': word, 'label': label})

  hasil_tanpaO = hasil.drop_duplicates().reset_index(drop=True)

  # hapus titik dan koma pada kolom text
  hasil['words'] = hasil['words'].str.replace('.com', '', regex=True)
  hasil['words'] = hasil['words'].str.replace('[.,()-:;"â]', '', regex=True)

  hasil['words'] = hasil['words'].str.lower()

  indeks = hasil[(hasil['label'] == 'B-PER') & (hasil['label'].shift(-1) != 'I-PER') & (hasil['words'].str.len() <= 3)].index
  hasil = hasil.drop(indeks).reset_index(drop=True)

  # print(hasil)

  for i in range(len(hasil)):
    try:
      if hasil['label'][i] == 'B-PER' and hasil['label'][i+1] != 'I-PER':
        people = hasil['words'][i]
      elif hasil['label'][i] == 'B-PER' and hasil['label'][i+1] == 'I-PER' and hasil['label'][i+2] != 'I-PER' and hasil['label'][i+3] != 'I-PER':
        people = hasil['words'][i] + ' ' + hasil['words'][i+1]
      elif hasil['label'][i] == 'B-PER' and hasil['label'][i+1] == 'I-PER' and hasil['label'][i+2] == 'I-PER' and hasil['label'][i+3] != 'I-PER':
        people = hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2]
      elif hasil['label'][i] == 'B-PER' and hasil['label'][i+1] == 'I-PER'and hasil['label'][i+2] == 'I-PER' and hasil['label'][i+3] == 'I-PER':
        people = hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2] + ' ' + hasil['words'][i+3]

    except:
      pass

    try:
      if hasil['label'][i] == 'B-PER' and i+1 == len(hasil):
        people = hasil['words'][i]
    except:
      pass

    try:
      if hasil['label'][i] == 'B-PER' and hasil['label'][i+1] == 'I-PER' and hasil['label'][i+2] != 'I-PER':
        people = hasil['words'][i] + ' ' + hasil['words'][i+1]
    except:
      pass

    try:
      if hasil['label'][i] == 'B-PER' and hasil['label'][i+1] == 'I-PER' and i+1+1 == len(hasil):
        people = hasil['words'][i] + ' ' + hasil['words'][i+1]
    except:
      pass

    try:
      if hasil['label'][i] == 'B-PER' and hasil['label'][i+1] == 'I-PER' and hasil['label'][i+2] == 'I-PER' and hasil['label'][i+3] != 'I-PER' and i+1+1 == len(hasil):
        people = hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2]
    except:
      pass

    try:
      if hasil['label'][i] == 'B-PER' and hasil['label'][i+1] == 'I-PER'and hasil['label'][i+2] == 'I-PER' and hasil['label'][i+3] == 'I-PER' and i+1+1+1 == len(hasil):
        people = hasil['words'][i] + ' ' + hasil['words'][i+1] + ' ' + hasil['words'][i+2] + ' ' + hasil['words'][i+3]
    except:
      pass

In [ ]:
hasil

,words,label
0,karena,O
1,itu,O
2,hasto,B-PER
3,menyayangkan,O
4,tindakan,O
5,pan,B-NOR
6,tersebut,O
7,dan,O


**Extract paragraf dari artikel**



In [ ]:
new_text = re.sub(r".*- ", "", text_mentah)
# print(new_text)

sentences = []
start = 0
for match in re.finditer(r'\n', new_text):
    end = match.start()
    sentences.append(new_text[start:end])
    start = match.end()

# add the last sentence
sentences.append(new_text[start:])
sentences.remove(sentences[-1])
print(sentences)
print(len(sentences))

['Sekjen DPP PDIP, Hasto Kristiyanto menyebut, dukungan PAN kepada Ganjar Pranowo merupakan bentuk pergeseran substansi demokrasi. Menurut Hasto, demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.', '', '“Banyak motif dalam mencalonkan seseorang. Ada yang bermotif untuk mendongkrak elektoral partai tersebut. Ada juga motif sebagai jalan pintas karena tidak melakukan kaderisasi di internal partai," kata Hasto di Sekolah Partai PDIP pada Kamis (2/3/2023).', '', 'Karena itu, Hasto menyayangkan tindakan PAN tersebut, dan berharap partai politik di Indonesia bisa mengusung kadernya sendiri. Sehingga pendidikan di internal partai berjalan dengan baik.', '', '“Tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling rasional," ujarnya.', '', 'Hasto juga mengklaim dalam proses dukungan Ganjar ketika pidato Ketua Umum PAN, Zulkifli Hasan dalam aca

In [ ]:
for i, y in enumerate(sentences):
  if len(y) < 10:
    sentences.remove(sentences[i])
  else:
    pass

for i, y in enumerate(sentences):
  print(i, '-', y)

0 - Sekjen DPP PDIP, Hasto Kristiyanto menyebut, dukungan PAN kepada Ganjar Pranowo merupakan bentuk pergeseran substansi demokrasi. Menurut Hasto, demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.
1 - “Banyak motif dalam mencalonkan seseorang. Ada yang bermotif untuk mendongkrak elektoral partai tersebut. Ada juga motif sebagai jalan pintas karena tidak melakukan kaderisasi di internal partai," kata Hasto di Sekolah Partai PDIP pada Kamis (2/3/2023).
2 - Karena itu, Hasto menyayangkan tindakan PAN tersebut, dan berharap partai politik di Indonesia bisa mengusung kadernya sendiri. Sehingga pendidikan di internal partai berjalan dengan baik.
3 - “Tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling rasional," ujarnya.
4 - Hasto juga mengklaim dalam proses dukungan Ganjar ketika pidato Ketua Umum PAN, Zulkifli Hasan dalam acara Rakorna

In [ ]:
sentence = sentences[3].lower()
sentence

'“tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling rasional," ujarnya.'

In [ ]:
  people = ''

  #menjelaskan
  match = re.search(r"\bmenjelaskan\b", sentence)
  if match:
    try:
      predict_person(sentence[:match.start()].strip())
      statement = sentence[match.end():].strip()

      if len(statement) < 20:
        statement = ''
        people = people

      print(people, "==>", statement)

    except:
      pass

## **dibawah ini udah mulai bisa FIX Combine NER**

### code statement

In [ ]:
generate_statement(sentences)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

 ==>  demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

 ==> “banyak motif dalam mencalonkan seseorang. ada yang bermotif untuk mendongkrak elektoral partai tersebut. ada juga motif sebagai jalan pintas karena tidak melakukan kaderisasi di internal partai," 


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

 ==> partai politik di indonesia bisa mengusung kadernya sendiri. sehingga pendidikan di internal partai berjalan dengan baik.
 ==> “tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling rasional,"


(['', '', '', '', ''],
 [' demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.',
  ', dukungan pan kepada ganjar pranowo merupakan bentuk pergeseran substansi demokrasi. menurut hasto, demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.',
  '“banyak motif dalam mencalonkan seseorang. ada yang bermotif untuk mendongkrak elektoral partai tersebut. ada juga motif sebagai jalan pintas karena tidak melakukan kaderisasi di internal partai," ',
  'partai politik di indonesia bisa mengusung kadernya sendiri. sehingga pendidikan di internal partai berjalan dengan baik.',
  '“tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling rasional,"'])

In [ ]:
def generate_statement(sentences):

  # coba generate statement pakai kalimat dari paragraf aja

  sentences = [x.lower() for x in sentences]
  people = ''
  full_people = []
  full_statement = []

  for sentence in sentences:

    try:

      #mengatakan
      match = re.search(r"\bmengatakan\b", sentence)
      if match:
        try:
          predict_person(sentence[:match.start()].strip())
          statement = sentence[match.end():].strip()

          if len(statement) < 20:
            statement = ''
            people = people

          print(people, "==>", statement)

          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #menjelaskan
      match = re.search(r"\bmenjelaskan\b", sentence)
      if match:
        try:
          predict_person(sentence[:match.start()].strip())
          statement = sentence[match.end():].strip()

          if len(statement) < 20:
            statement = ''

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #menurut
      match = re.search(r"menurut (.*?),", sentence)
      if match:
        try:
          statement = sentence.split(match.group(0))[1]

          if 'dia' not in match.group(1):
            predict_person(match.group(1))
          elif 'dia' in match.group(1):
            people = ''
            statement = ''

          if len(people) > 20:
            people = ''
            statement = ''
          else:
            pass

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #menurut
      match = re.search(r"\bmenurut\b", sentence)
      if match:
        try:
          predict_person(sentence[match.end():].strip())
          statement = sentence[match.end():].strip().split(people)[-1]

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #kata
      match = re.search(r"kata (.*)", sentence)
      if match:
        try:
          predict_person(match.group(1))

        except:
          pass

        statement = sentence.split(match.group(0))[0]
        if len(statement) < 20:
          try:
            statement = sentence.split(",", 1)[1]

          except:
            pass

        if len(statement) < 20:
          statement = ''
        else:
          pass

        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #jelas
      match = re.search(r"\bjelas\b", sentence)
      if match:
        try:
          predict_person(sentence[match.end():].strip())
          statement = sentence[:match.start()].strip()

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #ujarnya
      match = re.search(r"(.*?) ujarnya\.", sentence)
      if match:
        # people = people.mode()
        people = people
        statement = match.group(1)
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #sebaliknya
      match = re.search(r"sebaliknya,(.*)", sentence)
      if match:
        # people = people.mode()
        after_keyword = match.group(1)
        statement = after_keyword.split(",", 1)[1]
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #ucap
      match = re.search(r"ucap (.*)", sentence)
      if match:
        if 'dia' not in match.group(1).split(" ")[0]:
          people = match.group(1).split(" ")[0]
        elif 'dia' in match.group(1).split(" ")[0]:
          people = people

        if people.endswith("."):
          people = people[:-1]
        statement = sentence.split(match.group(0))[0]
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #tuturnya
      match = re.search(r"(.*?) tuturnya\.", sentence)
      if match:
        # people = people.mode()
        statement = match.group(1)
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #lanjut
      match = re.search(r"\blanjut\b (.*?),", sentence)
      if match:
          people = match.group(1)
          statement = sentence.split(people)[1]
          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

      #menegaskan
      match = re.search(r"\bmenegaskan\b", sentence)
      if match:
        try:
          predict_person(sentence[:match.start()].strip())
          statement = sentence[match.end():].strip()

          if len(statement) < 20:
            statement = ''
            people = people

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #menuturkan
      match = re.search(r"menuturkan (.*)", sentence)
      if match:
        statement = match.group(1)
        people = sentence.split("menuturkan")[0]
        if "," in people:
          people = people.split(",", 1)[1]
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #tutur
      match = re.search(r"\btutur\b", sentence)
      if match:
        try:
          predict_person(sentence[match.end():].strip())
          statement = sentence[:match.start()].strip()

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #menurutnya
      match = re.search(r"menurutnya,", sentence)
      if match:
        start_index = match.end()
        end_index = len(sentence)
        statement = sentence[start_index:end_index].strip()
        try:
          predict_person(sentence[start_index-50:start_index].strip())
        except:
          pass
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #ungkap
      match = re.search(r"\bungkap\b", sentence)
      if match:
        try:
          predict_person(sentence[match.end():].strip())
          statement = sentence[:match.start()].strip()

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #imbuh
      match = re.search(r"\bimbuh\b", sentence)
      if match:
        try:
          predict_person(sentence[match.end():].strip())
          statement = sentence[:match.start()].strip()

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #ujar
      match = re.search(r"\bujar\b", sentence)
      if match:
        try:
          predict_person(sentence[match.end():].strip())
          if len(people) > 2:
            statement = sentence[:match.start()].strip()

            print(people, "==>", statement)
            full_people.append(people)
            full_statement.append(statement)

          else:
            pass

        except:
          pass

      # #mengungkapkan
      # match = re.search(r"(?<=mengungkapkan ).+?(?=\.)", sentence)
      # if match:
      #   people = sentence[:match.start()].replace("mengungkapkan", "").strip()
      #   statement = match.group().strip()
      #   print(people, "==>", statement)

      #mengungkapkan
      match = re.search(r"\bmengungkapkan\b", sentence)
      if match:
        try:
          predict_person(sentence[:match.start()].strip())
          statement = sentence[match.end():].strip()

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #tutup
      match = re.search(r"\btutup\b (.*)", sentence)
      if match:
        words = match.group(1).strip()
        if len(words.split()) == 1:
          people = words.split()[0]
        else:
          people = people

        if '.' in people:
          split_titik = people.split(".")
          people = split_titik[0].strip()

        statement = sentence.split(match.group(0))[0]
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #sambungnya
      match = re.search(r"sambungnya", sentence)
      if match:
        people = people
        statement = sentence.split(match.group(0))[0].strip()

        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #tambahnya
      match = re.search(r"tambahnya", sentence)
      if match:
        people = people
        statement = sentence.split(match.group(0))[0].strip()

        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #pungkas
      match = re.search(r"\bpungkas\b", sentence)
      if match:
        people = sentence.split(match.group(0))[1].strip()
        statement = sentence.split(match.group(0))[0].strip()

        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #katanya
      match = re.search(r"katanya", sentence)
      if match:
        people = people
        statement = sentence.split(match.group(0))[0].strip()
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #pungkasnya
      match = re.search(r"pungkasnya", sentence)
      if match:
        people = people
        statement = sentence.split(match.group(0))[0].strip()

        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #menuturkan
      match = re.search(r'([a-zA-Z]+(?:\s+[a-zA-Z]+)?)\s+menuturkan,\s+(.*)', sentence)
      if match:
        if 'pun' in match.group(1):
          people = people

        else:

          if match.group(1) == 'ia':
            people = people
          else:
            people = match.group(1)
            if len(people.split()) == 1:
              people = people.split()[0]

        statement = match.group(2)
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #beber
      match = re.search(r'(.*)\bbeber\b\s(\w+)\s(.*)', sentence)
      if match:
          statement = match.group(1).strip()
          people = match.group(2)
          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)
      else:
          pass

      #berharap
      match = re.search(r"\bberharap\b", sentence)
      if match:
        try:
          predict_person(sentence[:match.start()].strip())
          statement = sentence[match.end():].strip()

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #tandas
      match = re.search(r"\btandas\b", sentence)
      if match:
        index = match.start()
        statement = sentence[:index].strip()
        people = sentence[index+len("tandas"):].strip()
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)
      else:
        pass

      #sebut
      match = re.search(r"\bsebut\b", sentence)
      if match:
        index = match.start()
        statement = sentence[:index].strip()
        people = sentence[index+len("sebut"):].strip()
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)
      else:
        pass

      #menyebutkan
      match = re.search(r"\bmenyebutkan\b", sentence)
      if match:
        if sentence[:match.start()].strip() == 'ia':
          people = people
        else:
          people = sentence[:match.start()].strip()

        statement = sentence[match.end()+2:].strip()

        if len(people)>20:
          people = ''
          statement = ''

        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #imbuhnya
      match = re.search(r"imbuhnya", sentence)
      if match:
        people = people
        statement = sentence.split(match.group(0))[0].strip()

        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #tambah
      match = re.search(r"\btambah\b", sentence)
      if match:
        try:
          predict_person(sentence[match.end():].strip())
          if not hasil['label'].str.contains('B-PER').any():
            pass

          else:
            if len(people) > 2:
              statement = sentence[:match.start()].strip()
              print(people, "==>", statement)
              full_people.append(people)
              full_statement.append(statement)

            else:
              pass

        except:
          pass

      #ucapnya
      match = re.search(r"ucapnya", sentence)
      if match:
        people = people
        statement = sentence.split(match.group(0))[0].strip()

        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

      #menekankan
      match = re.search(r"\bmenekankan\b", sentence)
      if match:
        try:
          predict_person(sentence[:match.start()].strip())
          statement = sentence[match.end():].strip()

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #menyatakan
      match = re.search(r"\bmenyatakan\b", sentence)
      if match:
        try:
          predict_person(sentence[:match.start()].strip())
          statement = sentence[match.end():].strip()

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #menyebut
      match = re.search(r"\bmenyebut\b", sentence)
      if match:
        try:
          predict_person(sentence[:match.start()].strip())
          statement = sentence[match.end():].strip()

          print(people, "==>", statement)
          full_people.append(people)
          full_statement.append(statement)

        except:
          pass

      #lanjutnya
      match = re.search(r"\blanjutnya\b", sentence)
      if match:
        people = people
        statement = sentence.split(match.group(0))[0].strip()
        print(people, "==>", statement)
        full_people.append(people)
        full_statement.append(statement)

    except:
      pass

  return full_people, full_statement

In [ ]:
full_people

['hasto', 'hasto', 'hasto kristiyanto', 'hasto', 'hasto', 'hasto']

In [ ]:
full_statement

[' demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.',
 ', demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.',
 ', dukungan pan kepada ganjar pranowo merupakan bentuk pergeseran substansi demokrasi. menurut hasto, demokrasi yang baik adalah mendidik kader sendiri dan mengusungnya untuk pilpres dibandingkan mengambil kader partai lain demi meningkatkan suara.',
 '“banyak motif dalam mencalonkan seseorang. ada yang bermotif untuk mendongkrak elektoral partai tersebut. ada juga motif sebagai jalan pintas karena tidak melakukan kaderisasi di internal partai," ',
 'partai politik di indonesia bisa mengusung kadernya sendiri. sehingga pendidikan di internal partai berjalan dengan baik.',
 '“tetapi yang penting kita belajar bahwa pelembagaan partai itu sangat penting dan merupakan pilihan yang paling r

### kesini untuk hasil

In [ ]:
# coba stopwords di people agar string selain nama di remove

people_statement = {'result':
                    [
                        {'person' : 'jokowi',
                         'statement' : 'statement nya apa'},

                     {'person' : 'jokowi',
                      'statement' : 'statement nya apa'}

                     ]
                    }

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
new_people_statement = {}
# Menghapus stopwords dari kunci dictionary
for key, value in people_statement.items():
    words = key.split()
    filtered_words = [word for word in words if word not in stop_words]
    new_key = ' '.join(filtered_words)
    new_people_statement[new_key] = value

new_people_statement

{'result': [{'person': 'jokowi', 'statement': 'statement nya apa'},
  {'person': 'jokowi', 'statement': 'statement nya apa'}]}

## wiraindrak/entity-based-sentiment-analysis

In [ ]:
!pip install gradio

In [ ]:
from transformers import pipeline
import matplotlib.pyplot as plt
# import twitter_scraper as ts
import pandas as pd

import gradio as gr

pretrained_sentiment = "w11wo/indonesian-roberta-base-sentiment-classifier"
pretrained_ner = "cahya/bert-base-indonesian-NER"

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=pretrained_sentiment,
    tokenizer=pretrained_sentiment,
    return_all_scores=True
)

ner_pipeline = pipeline(
    "ner",
    model=pretrained_ner,
    tokenizer=pretrained_ner,
    grouped_entities=True
)

examples = [
    "Jokowi sangat kecewa dengan POLRI atas kerusuhan yang terjadi di Malang",
    "Lesti marah terhadap perlakuan KDRT yang dilakukan oleh Bilar",
    "Ungkapan rasa bahagia diutarakan oleh Coki Pardede karena kebabasannya dari penjara"
]

def sentiment_analysis(text):
    output = sentiment_pipeline(text)
    return {elm["label"]: elm["score"] for elm in output[0]}

def ner(text):
    output = ner_pipeline(text)
    for elm in output:
        elm['entity'] = elm['entity_group']
    return {"text": text, "entities": output}

def sentiment_ner(text):
    return sentiment_analysis(text), ner(text)

def sentiment_df(df):
    text_list = list(df["Text"].astype(str).values)
    result = [sentiment_analysis(text) for text in text_list]
    labels = []
    scores = []
    for pred in result:
        idx = list(pred.values()).index(max(list(pred.values())))
        labels.append(list(pred.keys())[idx])
        scores.append(round(list(pred.values())[idx], 3))
    df['Label'] = labels
    df['Score'] = scores
    return df

def ner_df(df):
    text_list = list(df["Text"].astype(str).values)
    label_list = list(df["Label"].astype(str).values)
    result = [ner(text) for text in text_list]
    terms = []
    sentiments = []
    ent = ['PER', 'NOR']
    for i, preds in enumerate(result):
        for pred in preds['entities']:
            if pred['entity_group'] in ent:
                terms.append(pred['word'])
                sentiments.append(label_list[i])
    df_ner = pd.DataFrame(columns=['Entity', 'Sentiment'])
    df_ner['Entity'] = terms
    df_ner['Sentiment'] = sentiments
    return df_ner


def twitter_analyzer(keyword, max_tweets):
    df = ts.scrape_tweets(keyword, max_tweets=max_tweets)
    df["Text"] = df["Text"].apply(ts.preprocess_text)
    df = sentiment_df(df)

    # df_ner = ner_df(df)
    # df_ner = df_ner[df_ner.Entity != keyword]

    fig = plt.figure()
    df.groupby(["Label"])["Text"].count().plot.pie(autopct="%.1f%%", figsize=(6,6))
    return fig, df[["URL", "Text", "Label", "Score"]]

if __name__ == "__main__":

    with gr.Blocks() as demo:

        gr.Markdown("""<h1 style="text-align:center">Tweet Analyzer - Indonesia</h1>""")

        gr.Markdown(
            """
            Creator: Wira Indra Kusuma
            """
            )

        with gr.Tab("Single Input"):
            with gr.Blocks():
                with gr.Row():
                    with gr.Column():
                        input_text = gr.Textbox(label="Input Text")
                        analyze_button = gr.Button(label="Analyze")
                        examples_bar = gr.Examples(examples=examples, inputs=input_text)
                    with gr.Column():
                        sent_output = gr.Label(label="Sentiment Analysis")
                        ner_output = gr.HighlightedText(label="Named Entity Recognition")

        with gr.Tab("Twitter"):
            with gr.Blocks():
                with gr.Row():
                    with gr.Column():
                        keyword_textbox = gr.Textbox(lines=1, label="Keyword")
                        max_tweets_component = gr.Number(value=10, label="Total of Tweets to Scrape", precision=0)
                        submit_button = gr.Button("Submit")

                    plot_component = gr.Plot(label="Pie Chart of Sentiments")
                dataframe_component = gr.DataFrame(type="pandas",
                                                label="Dataframe",
                                                max_rows=(20,'fixed'),
                                                overflow_row_behaviour='paginate',
                                                wrap=True)

                # df_ner = gr.DataFrame(type="pandas",
                #                                 label="Dataframe",
                #                                 max_rows=(20,'fixed'),
                #                                 overflow_row_behaviour='paginate',
                #                                 wrap=True)


        analyze_button.click(sentiment_ner, input_text, [sent_output, ner_output])
        submit_button.click(twitter_analyzer,
                    inputs=[keyword_textbox, max_tweets_component],
                    outputs=[plot_component, dataframe_component])

        gr.Markdown(
                """
                """

            )

    demo.launch(inbrowser=True)

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/pipelines/token_classification.py:159: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>